In [ ]:
""" MODELO PRE ENTRENADO EFFICIENT NET - FINE TUNING PCAM
Dataset: Patch Camelyon
Fuente del modelo: keras_hub
Modelo: Efficient net Keras_hub models preentreenada Imagenet 12K: efficientnet_b5_sw_imagenet
Entorno de los modelos: Keras Hub, tensorflow datasets
"""

' MODELO PRE ENTRENADO EFFICIENT NET - FINE TUNING PCAM\nDataset: Patch Camelyon\nFuente del modelo: keras_hub\nModelo: Efficient net Keras_hub models preentreenada Imagenet 12K: efficientnet_b5_sw_imagenet\nEntorno de los modelos: Keras Hub, tensorflow datasets\n'

In [ ]:
!pip install tensorflow --upgrade
!pip install keras_cv
!pip install keras_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 119.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0

In [ ]:
import tensorflow as tf
import keras
import keras_hub
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
# Conectar con Google Drive
from google.colab import drive
_ = drive.mount('/content/drive')
base_folder = "/content/drive/MyDrive/00 VIU/10 TFM"

Mounted at /content/drive


In [ ]:
# Leer datasets PCam desde copia en disco:
ruta_pcam_tr = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_train"
train_ds = tf.data.Dataset.load(ruta_pcam_tr)
ruta_pcam_val = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_val"
val_ds = tf.data.Dataset.load(ruta_pcam_val)

# Usar la función 'map' para dejar solamente la imagen y la etiqueta
# quitando la identificación de la imagen
train_ds = train_ds.map(lambda x: (x["image"], x["label"]), \
                        num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda x: (x["image"], x["label"]), \
                        num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
print("len(train_ds):",len(train_ds))
print("len(val_ds):",len(val_ds))

len(train_ds): 262144
len(val_ds): 32768


In [ ]:
#  Instancier el preprocesador de entrada para normalizar las imagenes al rango 0..1
preprocessor = keras.layers.Pipeline([keras.layers.Rescaling(1.0 / 255)])
# Preparar el dataset en lotes y usar prefetch para optimizar el entrenamiento
batch_size = 32
train_ds = train_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Instanciar el clasificador de imagenes basado en resnet
image_classifier = keras_hub.models.ImageClassifier.from_preset(
    "efficientnet_b5_sw_imagenet",
    activation="softmax",
    num_classes=2,
    preprocessor=preprocessor,
)
# Compilar el modelo:
image_classifier.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

100%|██████████| 2.04k/2.04k [00:00<00:00, 3.33MB/s]


100%|██████████| 5.36k/5.36k [00:00<00:00, 9.80MB/s]


100%|██████████| 202M/202M [00:13<00:00, 15.2MB/s]


In [ ]:
# Entrenar (ej: 3 epocas)
image_classifier.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
  )

Epoch 1/3
8192/8192 ━━━━━━━━━━━━━━━━━━━━ 1382s 149ms/step - accuracy: 0.9211 - loss: 0.2038 - val_accuracy: 0.8813 - val_loss: 0.3140
Epoch 2/3
8192/8192 ━━━━━━━━━━━━━━━━━━━━ 1167s 142ms/step - accuracy: 0.9674 - loss: 0.0959 - val_accuracy: 0.8739 - val_loss: 0.4481
Epoch 3/3
8192/8192 ━━━━━━━━━━━━━━━━━━━━ 1162s 142ms/step - accuracy: 0.9781 - loss: 0.0638 - val_accuracy: 0.8287 - val_loss: 0.7883


In [ ]:
# Cargar el dataset de prueba
ruta_pcam_test = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_test"
test_ds = tf.data.Dataset.load(ruta_pcam_test)
# Usar la función 'map' para dejar solamente la imagen y la etiqueta
# quitando la identificación de la imagen
test_ds = test_ds.map(lambda x: (x["image"], x["label"]), \
                        num_parallel_calls=tf.data.AUTOTUNE)
print("len(test_ds):",len(test_ds))
# Preparar el dataset en lotes y usar prefetch para optimizar la evaluación
test_ds = test_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

len(test_ds): 32768


In [ ]:
# Evaluar el modelo
_, accuracy = image_classifier.evaluate(test_ds)
print(f"Accuracy en el dataset de prueba: {accuracy}")

1024/1024 ━━━━━━━━━━━━━━━━━━━━ 43s 40ms/step - accuracy: 0.8182 - loss: 0.7394
Accuracy en el dataset de prueba: 0.817901611328125


In [ ]:
image_classifier.save_to_preset(base_folder+"/Modelos entrenados/efficient_net")